# All MEME Images

This generates the data for all gene tables for i-modulons in PRECISE 1.0 so that they can be plotted in the ModulomeVis site.

In [1]:
import sys
# be sure that this points to a clone of github.com/SBRG/ICA
sys.path.append('../../../ica/') 
from icaviz.plotting import *
DATA_DIR = '../../../ica/data/precise_data/'
GENE_DIR = '../../../ica/data/annotation/'
enrich = pd.read_csv(DATA_DIR+'curated_enrichments.csv', index_col = 0)
names = enrich['name'].tolist()
ica_data = load_data(X=DATA_DIR+'log_tpm.csv',
                     S=DATA_DIR+'S.csv',
                     A=DATA_DIR+'A.csv',
                     metadata=DATA_DIR+'metadata.csv',
                     annotation=GENE_DIR+'gene_info.csv',
                     trn=GENE_DIR+'TRN.csv',
                     fasta=GENE_DIR+'NC_000913.3.fasta',
                     cutoff = 550)
pd.set_option('display.max_rows', None)

In [10]:
from regulon_meme import *

## Code from Anand

In [2]:
# Force -> True if you are running this for the first time
force = False

In [4]:
sites = []
motifs = []

comp_names = enrich[enrich.Category!='Genomic Alterations'].index
# Run once using 300 nts upstream
for k in tqdm(comp_names):
    DF_motif,DF_sites = find_motifs(ica_data,k,nmotifs=1,verbose=True,evt=1e-3,
                                    force=force,upstream=300,downstream=0,maxw=30)
    
    # Save all found binding sites
    if len(DF_sites) > 0:
        tmp_sites = DF_sites[DF_sites.site_seq.notnull()]
        tmp_sites.index = pd.MultiIndex.from_tuples([(k, x[1]) for x in tmp_sites.index])
        sites.append(tmp_sites)

found_motifs1 = [k for k,v in ica_data.motif_info.items() if len(v[0])>0]

# Run again for remaining sequences with 600 nt upstream and 100 downstream
for k in tqdm(set(comp_names) - set(found_motifs1)):
    DF_motif,DF_sites = find_motifs(ica_data,k,nmotifs=1,verbose=False,evt=1e-3,
                                    force=force,upstream=600,downstream=100,maxw=30)
    
    # Save all found binding sites
    if len(DF_sites) > 0:
        tmp_sites = DF_sites[DF_sites.site_seq.notnull()]
        tmp_sites.index = pd.MultiIndex.from_tuples([(k, x[1]) for x in tmp_sites.index])
        sites.append(tmp_sites)
    
found_motifs2 = [k for k,v in ica_data.motif_info.items() if len(v[0])>0]

Finding motifs for 8 sequences


FileNotFoundError: [Errno 2] No such file or directory: 'motifs/0/meme.xml'

## Generate all CSVs

In [5]:
data_folder = 'data/'
for k, row in enrich.iterrows():
    this_fig = gene_table_df(ica_data, k, row) #replace function name
    this_fig.to_csv(data_folder + '%i_gene_table.csv'%(k)) #replace file suffix
    